In [ ]:
import pandas as pd
from openbabel import pybel
import os
from pymol import cmd
import py3Dmol
from rdkit import Chem
from rdkit.Chem import AllChem
from joblib import Parallel, delayed
from tqdm import tqdm
import glob
import sys, os, random
sys.path.insert(1,'/your/pathway/for/Jupyter_Dock-main/Jupyter_Dock-main/utilities/')
from my_utils import getbox, generate_ledock_file, dok_to_sdf
import warnings
warnings.filterwarnings("ignore")

%config Completer.use_jedi = False
import subprocess

### The following code should be executed separately for active,inactive and decoy molecularity respectively.

# Generate mol2 files

In [ ]:
molecules=pd.read_csv('/pathway/for/activate_inactive_decoys.csv/')
smiles=molecules['smiles'].tolist()

In [ ]:
for index,smi in enumerate(molecules):
    mol=pybel.readstring(string=smi,format='smiles')
    mol.title='active_'+str(index) #mark the correct prefix
    mol.make3D('mmff94s')
    mol.localopt(forcefield='mmff94s', steps=500)
    output_folder='/the/output/folder/pathway/smiles_mol2/'
    os.makedirs(output_folder,exist_ok=True)
    out=pybel.Outputfile(filename=output_folder+mol.title+'.mol2',format='mol2',overwrite=True)
    out.write(mol)
    out.close()

# Generic functions

### Preparate the docking files

In [ ]:
receptor='/your/target/file/pathway/protein.pdb'
output_folder='/smina/output/dockded/'
os.makedirs(output_active_folder,exist_ok=True)
smile_folder='/the/output/folder/pathway/smiles_mol2/'

### Smina

In [ ]:
def run_smina(lig):
    input_file = os.path.join(smile_folder,lig)
    cmd.load(filename=receptor,format='pdb',object='prot')
    cmd.cmd.load(filename=input_file,format='mol2',object='lig')
    center,size=getbox(selection='lig',extending=8,software='vina')
    cmd.delete('all')
    output_folder=output_folder
    molecular_index = os.path.splitext(lig)[0]
    output_file =os.path.join(output_folder ,f'{molecular_index}_docked.sdf')
    smina_command= f"/your/pathway/for/bin/smina --ligand {input_file} --receptor {receptor} --out {output_file} --center_x {center['center_x']} --center_y {center['center_y']} --center_z {center['center_z']} --size_x {size['size_x']} --size_y {size['size_x']} --size_z {size['size_x']} --num_modes 1 --exhaustiveness 8 --seed 0"
    subprocess.run(smina_command, shell=True)

In [ ]:
molecular_files=[f for f in os.listdir(smile_folder) if f.endswith(".mol2")]

In [ ]:
%%time
Parallel(n_jobs = 40, backend = 'multiprocessing')(delayed(run_smina)(lig) for lig in tqdm(molecular_files))

### CNN-Score

In [ ]:
def run_cnn(lig):
    input_file = os.path.join(smile_folder,lig)
    output_folder=output_folder
    molecular_index = os.path.splitext(lig)[0]
    output_file =os.path.join(output_folder ,f'{molecular_index}_docked.txt')
    cnn_command= f"/home/pathway/gnina --score_only -r {receptor} -l {input_file} > {output_file}"
    subprocess.run(cnn_command, shell=True)

In [ ]:
molecular_files=[f for f in os.listdir(smile_folder) if f.endswith(".mol2")]

In [ ]:
%%time
Parallel(n_jobs = 40, backend = 'multiprocessing')(delayed(run_cnn)(lig) for lig in tqdm(molecular_files))

### RF-SCORE-VS

In [ ]:
def run_rfscore(lig):
    input_file = os.path.join(smile_folder,lig)
    output_folder=output_folder
    molecular_index = os.path.splitext(lig)[0]
    output_file =os.path.join(output_folder ,f'{molecular_index}_docked.sdf')
    rfscore_command= f"/home/pathway/rf-score-vs --receptor {receptor} {input_file} -O {output_file}"
    subprocess.run(rfscore_command, shell=True)

In [ ]:
molecular_files=[f for f in os.listdir(smile_active_folder) if f.endswith(".mol2")]

In [ ]:
%%time
Parallel(n_jobs = 40, backend = 'multiprocessing')(delayed(run_rfscore)(lig) for lig in tqdm(molecular_files))